In [50]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import numpy as np
import pandas as pd

In [51]:
df=pd.read_csv('input.csv')

In [52]:
df.head()

,intent,text,responses,extension,context,entityType,entities
0,Greeting,Hi,"['Hi human, please tell me your GeniSys user',...","{'function': '', 'entities': False, 'responses...","{'in': '', 'out': 'GreetingUserRequest', 'clea...",NaN,[]
1,Greeting,Hi there,"['Hi human, please tell me your GeniSys user',...","{'function': '', 'entities': False, 'responses...","{'in': '', 'out': 'GreetingUserRequest', 'clea...",NaN,[]
2,Greeting,Hola,"['Hi human, please tell me your GeniSys user',...","{'function': '', 'entities': False, 'responses...","{'in': '', 'out': 'GreetingUserRequest', 'clea...",NaN,[]
3,Greeting,Hello,"['Hi human, please tell me your GeniSys user',...","{'function': '', 'entities': False, 'responses...","{'in': '', 'out': 'GreetingUserRequest', 'clea...",NaN,[]
4,Greeting,Hello there,"['Hi human, please tell me your GeniSys user',...","{'function': '', 'entities': False, 'responses...","{'in': '', 'out': 'GreetingUserRequest', 'clea...",NaN,[]


In [53]:
df.shape

(143, 7)

In [72]:
random_samples = df.sample(n=int(df.shape[0]/10))

In [73]:
random_samples.shape

(14, 7)

In [74]:
val_x,val_y=random_samples.text,random_samples.intent

In [54]:
x=df.text
y=df.intent

In [55]:
from sklearn.model_selection import train_test_split

In [56]:
x[:5]

0             Hi
1       Hi there
2           Hola
3          Hello
4    Hello there
Name: text, dtype: object

In [57]:
y[:5]

0    Greeting
1    Greeting
2    Greeting
3    Greeting
4    Greeting
Name: intent, dtype: object

In [58]:
x_train,x_test, y_train,y_test= train_test_split(x,y,test_size=0.2,random_state=10)

In [59]:
import tensorflow as tf

In [60]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(x_train)

In [61]:
train_sequences = tokenizer.texts_to_sequences(x_train)

In [62]:
test_sequences = tokenizer.texts_to_sequences(x_test)

In [75]:
val_sequences = tokenizer.texts_to_sequences(val_x)

In [76]:
# Pad sequences to ensure uniform length
max_length = max(len(seq) for seq in train_sequences + test_sequences)
train_padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=max_length, padding='post')
test_padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=max_length, padding='post')
val_padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(val_sequences, maxlen=max_length, padding='post')

In [77]:
# Convert intents to unique integer labels
intent_label_mapping = {label: idx for idx, label in enumerate(y_train.unique())}
y_train_labels = y_train.map(intent_label_mapping)
y_test_labels= y_test.map(intent_label_mapping)
val_labels=val_y.map(intent_label_mapping)

In [78]:
# Convert integer labels to one-hot encoding
num_intents = len(intent_label_mapping)
train_intent_labels = tf.keras.utils.to_categorical(y_train_labels, num_classes=num_intents)
test_intent_labels = tf.keras.utils.to_categorical(y_test_labels, num_classes=num_intents)
val_intent_labels = tf.keras.utils.to_categorical(val_labels, num_classes=num_intents)

In [85]:
# Define RNN model with additional layers and increased units in the embedding layer
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=200, input_length=max_length),
    tf.keras.layers.LSTM(128, return_sequences=True),  # Adding another LSTM layer with return_sequences=True
    tf.keras.layers.LSTM(64),  # Adding another LSTM layer
    tf.keras.layers.Dense(128, activation='relu'),  # Adding a Dense hidden layer
    tf.keras.layers.Dense(num_intents, activation='softmax')  # Output layer with softmax activation
])

In [86]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [87]:
model.fit(train_padded_sequences, train_intent_labels, epochs=50, batch_size=32, validation_data=(val_padded_sequences, val_intent_labels))

Epoch 1/50
4/4 [==============================] - 5s 520ms/step - loss: 3.0913 - accuracy: 0.0702 - val_loss: 3.0844 - val_accuracy: 0.1429
Epoch 2/50
4/4 [==============================] - 0s 22ms/step - loss: 3.0808 - accuracy: 0.1930 - val_loss: 3.0755 - val_accuracy: 0.2857
Epoch 3/50
4/4 [==============================] - 0s 21ms/step - loss: 3.0679 - accuracy: 0.2544 - val_loss: 3.0585 - val_accuracy: 0.1429
Epoch 4/50
4/4 [==============================] - 0s 21ms/step - loss: 3.0452 - accuracy: 0.1930 - val_loss: 3.0303 - val_accuracy: 0.0714
Epoch 5/50
4/4 [==============================] - 0s 21ms/step - loss: 2.9987 - accuracy: 0.1930 - val_loss: 2.9745 - val_accuracy: 0.0714
Epoch 6/50
4/4 [==============================] - 0s 26ms/step - loss: 2.8916 - accuracy: 0.1930 - val_loss: 2.8666 - val_accuracy: 0.0714
Epoch 7/50
4/4 [==============================] - 0s 21ms/step - loss: 2.7070 - accuracy: 0.2105 - val_loss: 2.7174 - val_accuracy: 0.0714
Epoch 8/50
4/4 [==========

In [88]:
model.evaluate(test_padded_sequences, test_intent_labels)

1/1 [==============================] - 0s 38ms/step - loss: 2.3704 - accuracy: 0.4483


[2.370412588119507, 0.4482758641242981]